In [1]:
import numpy as np
import pandas as pd
import os
import re
import pathlib

import src.SimData  as sim_data

In [2]:
###* Experimental Data CO 2

schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "fracCO2ini_relO2": "frac_CO2",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "Tgas":         "Tgas",
    "O mean":       "O",
    "CO mean":      "CO",
    "MeanLossProb": "gamma_exp"
}

file_path = "Experimental_data_CO/Data_CO2_TD.xlsx"

pd_exp = pd.read_excel(file_path)
pd_result_exp = pd.DataFrame()

for old, new in schema.items():
    if old in pd_exp.columns:
        pd_result_exp[new] = pd_exp[old]


pd_result_exp["Tw"] = pd_result_exp["Tw"] + 273.15 
# pd_result_exp["current"] = pd_result_exp["current"] * 1e-3
pd_result_exp["Tnw"] = pd_result_exp["Tw"] + 0.45 * (pd_result_exp["Tgas"] - pd_result_exp["Tw"])
pd_result_exp["frac_O2"] = 1.0 - pd_result_exp["frac_CO2"]

print(pd_result_exp.head())
print(pd_result_exp.shape)

   pressure  current  frac_CO2      Tw        Tgas  gamma_exp         Tnw  \
0       0.4       40       0.0  323.15  360.560032   0.001819  339.984514   
1       0.6       40       0.0  323.15  372.733684   0.000995  345.462658   
2       0.8       40       0.0  323.15  384.630566   0.000841  350.816255   
3       1.0       40       0.0  323.15  396.252108   0.000775  356.045949   
4       1.5       40       0.0  323.15  424.110735   0.000827  368.582331   

   frac_O2  
0      1.0  
1      1.0  
2      1.0  
3      1.0  
4      1.0  
(332, 8)


In [3]:

folder_path = "Experimental_data_CO/ResultsLoKIV2/"
file_name_list = os.listdir(folder_path)
nb_list = sorted([int(file.split('_')[2]) for file in file_name_list])


def _read_output_densities(output_densities_path, skiprows=5):
        
    parts = []
    for line in output_densities_path.read_text().splitlines()[skiprows:]:
        toks = [tok for tok in line.split() if tok]
        if len(toks) < 2:
            continue
        elif toks[0] == '|':
            continue
            
        parts.append(toks)
        
    max_len = max(len(p) for p in parts)
    full_rows = [p for p in parts if len(p) == max_len]
        
    arr = np.array(full_rows, dtype=object)
    names = arr[:, 0].tolist()
    values = arr[:, 1].astype(float)

    ##! Fixed for now
    O2_index = [0, 1, 2, 3, 4]
    O_index = [5, 6, 7, 8]
    O3_index = [9, 10]
    CO2_index = [11, 12, 13]
    CO_index = [14, 15, 16]
    C_index = [17]
    
    O2_den = np.sum(values[O2_index])
    O_den = np.sum(values[O_index])
    O3_den = np.sum(values[O3_index])
    CO2_den = np.sum(values[CO2_index])
    CO_den = np.sum(values[CO_index])
    C_den = np.sum(values[C_index])
    
    vec_den = [O2_den, O_den, O3_den, CO2_den, CO_den, C_den]
    names = ['O2', 'O', 'O3', 'CO2', 'CO', 'C']
    
    print("vec_den: ", vec_den)
    
    return vec_den, names


nb_cols = 6
data_LoKI = np.zeros((pd_result_exp.shape[0],nb_cols), dtype=float)


for counter in range(pd_result_exp.shape[0]):
    if counter in nb_list:
        
        try: 
            file_name = folder_path + f"sim_final_{counter}/chemFinalDensities.txt"
            out_path = pathlib.Path(file_name)
            vec_den, names = _read_output_densities(out_path)
        except:
            vec_den = [0] * nb_cols
            
        data_LoKI[counter, :] = vec_den
    else:
        pass



vec_den:  [9.75357337484629e+21, 1.0832360780108205e+21, 5.867946763204064e+17, 0.0, 0.0, 0.0]
vec_den:  [1.3496126602135472e+22, 2.3167789356169804e+21, 3.827035508334344e+17, 0.0, 0.0, 0.0]
vec_den:  [1.7077656603528861e+22, 3.184974465294668e+21, 3.8581423038119226e+17, 0.0, 0.0, 0.0]
vec_den:  [2.0653777757060353e+22, 3.923748625854419e+21, 4.170786245471381e+17, 0.0, 0.0, 0.0]
vec_den:  [2.9620742619759123e+22, 4.895195861707167e+21, 5.139531517901761e+17, 0.0, 0.0, 0.0]
vec_den:  [3.844593727264055e+22, 5.545968340643622e+21, 6.014361610935808e+17, 0.0, 0.0, 0.0]
vec_den:  [5.368190499089881e+22, 6.358394244219105e+21, 7.413410075079749e+17, 0.0, 0.0, 0.0]
vec_den:  [8.047039554492545e+22, 7.002461300724753e+21, 9.277984474485166e+17, 0.0, 0.0, 0.0]
vec_den:  [1.0283013869982286e+22, 1.093668645010195e+21, 5.685151908897414e+17, 0.0, 0.0, 0.0]
vec_den:  [1.4659250667658127e+22, 1.9474669406865604e+21, 5.006686579626174e+17, 0.0, 0.0, 0.0]
vec_den:  [1.9011347972403715e+22, 2.6737

In [4]:
print(names)
print(nb_list)

['O2', 'O', 'O3', 'CO2', 'CO', 'C']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 120, 121, 122, 130, 180, 181, 182, 183, 184, 185, 187, 230, 280, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316]


In [5]:
print(data_LoKI)

[[9.75357337e+21 1.08323608e+21 5.86794676e+17 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.34961266e+22 2.31677894e+21 3.82703551e+17 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.70776566e+22 3.18497447e+21 3.85814230e+17 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [6]:
print(names)

['O2', 'O', 'O3', 'CO2', 'CO', 'C']


In [7]:

print(pd_result_exp.columns)

for idx, name in enumerate(names):
    pd_result_exp[name] = data_LoKI[:, idx] * 1e-6
    # pd_result_exp[name] = data_LoKI[:, idx]


mask = (pd_result_exp[names] != 0).all(axis=1)
pd_result_clean_exp = pd_result_exp.loc[mask].reset_index(drop=True)

pd_result_clean_exp['Tw'] = pd_result_clean_exp['Tw'] - 273.15

path_excel = "Experimental_data_CO/Data_CO2_TD_LoKIV2.xlsx"
pd_result_clean_exp.to_excel(path_excel)

print(pd_result_clean_exp.head())




Index(['pressure', 'current', 'frac_CO2', 'Tw', 'Tgas', 'gamma_exp', 'Tnw',
       'frac_O2'],
      dtype='object')
   pressure  current  frac_CO2    Tw        Tgas  gamma_exp         Tnw  \
0       1.5       40       1.0  50.0  555.183313   0.000317  427.564991   
1       2.0       40       1.0  50.0  606.771416   0.000274  450.779637   
2       3.0       40       1.0  50.0  680.737954   0.000297  484.064579   
3       5.0       40       1.0  50.0  757.395050   0.000317  518.560272   
4       0.8       20       1.0  50.0  419.735699   0.000175  366.613565   

   frac_O2            O2             O            O3           CO2  \
0      0.0  9.435162e+15  1.396905e+15  5.198078e+11  1.061617e+16   
1      0.0  1.234038e+16  1.830638e+15  8.764867e+11  1.422151e+16   
2      0.0  1.811999e+16  2.070531e+15  1.522967e+12  2.217275e+16   
3      0.0  2.867847e+16  2.395379e+15  2.778697e+12  3.938798e+16   
4      0.0  5.333158e+15  7.027822e+14  2.930450e+11  6.019152e+15   

           

In [8]:

### store in buffer data in hdf5 style

schema_buffer = {
    "pressure":     "pressure",
    "current":      "current",
    "frac_CO2":     "frac_CO2",
    "Tgas":         "Tgas",
    "Tnw":          "Tnw", 
    "Tw":           "Tw",
    "O":            "O",
    "O2":           "O2",
    "O3":           "O3",
    "C":            "C",
    "CO":           "CO",
    "CO2":          "CO2",
    "gamma_exp":    "gamma_exp"
}

output_file = "Experimental_data_CO2_TD_LoKIV2.hdf5"
file_path = [path_excel]

data_loader = sim_data.DataLoader(schema_buffer, output_file=output_file, files_path=file_path)
data_loader.load_data(force_update=True)